In [ ]:
from pd2image import convert_from_path
import pandas as pd

### Get path to pdf and start conversion process

In [ ]:
pdfs = r'"C:\User\Path_to_PDF.pdf"'
pages = convert_from_path(pdfs, 350)

Convert each page on pdf document to jpg image

In [ ]:
i = 1
for page in pages:
    image_name = "Page_" + str(i) + ".jpg"
    page.save(image_name, "JPEG")
    i = i+1

In [ ]:
import cv2
import matplotlib.pyplot as plt

### Select image pages of PDF converted to image and apply above method

In [ ]:
FILENAME = "C\User\Path_to_image_Page_1.jpg"
image = cv2.imread(FILENAME,0)
plot1 = plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

In [ ]:
thresh,img_bin = cv2.threshold(img,128,255,cv2.THRESH_BINARY)
img_bin = 255 - img_bin
plotting = plt.imshow(img_bin,cmap='gray')
plt.title("Inverted Image with global thresh holding")
plt.show()

In [ ]:
img_bin1 = 255 - img
thresh1,img_bin1_otsu = cv2.threshold(img_bin1,128,255,cv2.THRESH_OTSU)
plotting = plt.imshow(img_bin1_otsu,cmap='gray')
plt.title("Inverted Image with otsu thresh holding")
plt.show()

In [ ]:
img_bin2 = 255 - img
thresh1,img_bin_otsu = cv2.threshold(img_bin2,128,255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)
plotting = plt.imshow(img_bin_otsu,cmap='gray')
plt.title("Inverted Image with otsu thresh holding")
plt.show()

In [ ]:
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2,2))
print(kernel)

In [ ]:
import numpy as np
plt.figure(figsize = (30,30))

vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, np.array(img).shape[1]//100))
eroded_image = cv2.erode(img_bin_otsu, vertical_kernel,iterations=3)
plt.subplot(151),plt.imshow(eroded_image, cmap = 'gray')
plt.title('Image after erosion with vertical kernel'), plt.xticks([]), plt.yticks([])

vertical_lines = cv2.dilate(eroded_image, vertical_kernel, iterations=3)
plt.subplot(152),plt.imshow(vertical_lines, cmap='gray')
plt.title('Image after dilation with vertical kernel'), plt.xticks([]), plt.yticks([])

plt.show()

In [ ]:
plt.figure(figsize = (30,30))

hor_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (np.array(img).shape[1]//100, 1))
horizontal_lines = cv2.erode(img_bin, hor_kernel, iterations=5)
plt.subplot(153),plt.imshow(horizontal_lines, cmap='gray')
plt.title('Image after erosion with horizontal kernel'), plt.xticks([]), plt.yticks([])

horizontal_lines = cv2.dilate(horizontal_lines, hor_kernel, iterations=5)
plt.subplot(153),plt.imshow(horizontal_lines, cmap='gray')
plt.title('Image after erosion with horizontal kernel'), plt.xticks([]), plt.yticks([])

plt.show()

In [ ]:
plt.figure(figsize= (30,30))

vertical_horizontal_lines = cv2.addWeighted(vertical_lines, 0.5, horizontal_lines, 0.5,0.0)
vertical_horizontal_lines = cv2.erode(~vertical_horizontal_lines, kernel, iterations=3)
plt.subplot(151),plt.imshow(vertical_horizontal_lines,cmap='gray')
plt.title('Erosion'), plt.xticks([]), plt.yticks([])

thresh, vertical_horizontal_lines = cv2.threshold(vertical_horizontal_lines,128,255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
plt.subplot(152),plt.imshow(vertical_horizontal_lines,cmap='gray')
plt.title('global and otsu threshold'), plt.xticks([]), plt.yticks([])

bitxor = cv2.bitwise_xor(img,vertical_horizontal_lines)
plt.subplot(153),plt.imshow(bitxor,cmap='gray')
plt.title('Horizontal and vertical lines image bitxor'), plt.xticks([]), plt.yticks([])

bitnot = cv2.bitwise_not(bitxor)
plt.subplot(154),plt.imshow(bitxor,cmap='gray')
plt.title('Horizontal and vertical lines image with bitnot'), plt.xticks([]), plt.yticks([])

plt.show()

In [ ]:
### Import PyTesseract OCR Engine

In [ ]:
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C\User\Path_to_tesseract.exe'

In [ ]:
contours, hierachy = cv2.findContours(vertical_horizontal_lines, cv2.RET_TREE,cv2.CHAIN_APPROX_SIMPLE)

In [ ]:
boundingBoxes = [cv2.boundingRect(contour) for contour in contours]
(contours, boundingBoxes) = zip(*sorted(zip(contours,boundingBoxes),key=lambda x:x[1][1]))

In [ ]:
boxes = []
for contour in contours:
    x,y,w,h = cv2.boundingRect(contour)
    if(w<1000 and h<500):
        image = cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)
        boxes.append([x,y,w,h])
plotting = plt.imshow(image,cmap='gray')
plt.title("Identified contours")
plt.show()

In [ ]:
rows = []
columns = []
heights = [boundingBoxes[i][3] for i in range(len(boundingBoxes))]
mean = np.mean(heights)
print(mean)
columns.append(boxes[0])
previous=boxes[0]
for i in range(1,len(boxes)):
    if(boxes[i][1]<=previous[1]+mean/2):
        columns.append(boxes[i])
        previous=boxes[i]
        if(i==len(boxes)-1):
            rows.append(columns)
        else:
            rows.append(columns)
            columns=[]
            previous = boxes[i]
            columns.append(boxes[i])
print("Rows")
for row in rows:
    print(row)

In [ ]:
total_cells = 0
total_cells=len(row)
for i in range(len(row)):
    #print(row[i])
    if len(row[i]) > total_cells:
        total_cells = len(row[i])
        print(total_cells)
print(total_cells)

In [ ]:
center = [int(rows[i][j][0]+rows[i][j][2]/2) for j in range(len(rows[i])) if rows[0]]
print(center)

In [ ]:
center=np.array(center)
center.sort()
print(center)

In [ ]:
boxes_list = []
for i in range(len(rows)):
    l = [[] for _ in range(total_cells)] #Create list of empty sublists
    for j in range(len(rows[i])):
        diff = abs(center - (rows[i][j][0] + rows[i][j][2] / 4))
        minimum = min(diff)
        indexing = list(diff).index(minimum)
        while indexing >= len(l): #Appending empty sublists until indexing is valid
            l.append([])
        l[indexing].append(rows[i][j])
    boxes_list.append(l)
for box in boxes_list:
    print(box)

In [ ]:
dataframe_final = []
for i in range(len(boxes_list)):
    for j in range(len(boxes_list[i])):
        s=''
        if(len(boxes_list[i][j])==0):
            dataframe_final.append(' ')
        else:
            for k in range(len(boxes_list[i][j])):
                y,x,w,h = boxes_list[i][j][k][0],boxes_list[i][j][k][1], boxes[i][j][k][2],boxes_list[i][j][k][3]
                roi = bitnot[x:x+h, y:y+w]
                kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2,1))
                border = cv2.copyMakeBorder(roi,2,2,2,2, cv2.BORDER_CONSTANT,value=[255,255])
                resizing = cv2.resize(border, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
                dilation = cv2.dilate(resizing,kernel,iterations=1)
                erosion = cv2.erode(dilation,kernel,iterations=2)
                out = pytesseract.image_to_string(erosion)
                if(len(out)==0):
                    out = pytesseract.image_to_string(erosion)
                s = s+" "+ out
            dataframe_final.append(s)
print(dataframe_final)

In [ ]:
arr = np.array(dataframe_final)
arr

In [ ]:
new_rows = len(arr)//total_cells

In [ ]:
import pandas as pd
dataframe = pd.DataFrame(arr.reshape(new_rows, total_cells))
data = dataframe.style.set_properties(align="left")
#print(data)
#print(dataframe)
d=[]
for i in range(dataframe.shape[0]):
    for j in range(dataframe.shape[1]):
        print(dataframe.iloc[i,j],end=" ")
    print()

In [ ]:
print(dataframe)

In [ ]:
finaldf=dataframe
finaldf.head()

In [ ]:
### Can perform the necessary data cleaning before exporting data to excel

In [ ]:
finaldf.to_excel("firstTest.xlsx", index=False)